In [6]:
import os
import csv

# Fungsi untuk membaca struktur tabel dari file CSV dan menghasilkan pernyataan SQL CREATE TABLE
def generate_sql_create_table(csv_file):
    # Membaca nama file tanpa ekstensi
    table_name = os.path.splitext(os.path.basename(csv_file))[0]
    
    # Membaca struktur tabel dari file CSV
    with open(csv_file, 'r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        # Menggunakan struktur kolom pertama dalam CSV sebagai referensi untuk struktur tabel
        columns = reader.fieldnames
        # Menginisialisasi tipe data kolom dengan VARCHAR sebagai default
        column_types = {column: 'VARCHAR(255)' for column in columns}
        # Memeriksa sampel data untuk menentukan tipe data kolom yang sesuai
        sample_data = next(reader)
        for column, data in sample_data.items():
            # Jika data dapat diubah menjadi integer tanpa desimal, itu adalah tipe data INTEGER
            try:
                int_data = int(data)
                if int_data == float(data):
                    column_types[column] = 'INTEGER'
            except ValueError:
                pass
            # Jika data dapat diubah menjadi float, itu adalah tipe data DECIMAL
            try:
                float_data = float(data)
                if '.' in data:
                    column_types[column] = 'DECIMAL(10, 2)'
            except ValueError:
                pass
    
    # Membuat pernyataan SQL CREATE TABLE
    sql_create_table = f"CREATE TABLE IF NOT EXISTS {table_name} (\n"
    for column, data_type in column_types.items():
        sql_create_table += f"    {column} {data_type},\n"
    # Menghapus koma ekstra di akhir
    sql_create_table = sql_create_table.rstrip(',\n')
    sql_create_table += "\n);\n"
    
    return sql_create_table

# Fungsi untuk membaca data dari file CSV dan menghasilkan pernyataan SQL INSERT
def generate_sql_insert_from_csv(csv_file):
    # Membaca nama file tanpa ekstensi
    table_name = os.path.splitext(os.path.basename(csv_file))[0]
    
    # Membaca data dari file CSV
    with open(csv_file, 'r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        rows = [row for row in reader]
    
    # Membuat pernyataan SQL INSERT
    sql_inserts = []
    for row in rows:
        columns = ', '.join(row.keys())
        values = ', '.join([f"'{value}'" for value in row.values()])
        sql_insert = f"INSERT INTO {table_name} ({columns}) VALUES ({values});"
        sql_inserts.append(sql_insert)
    
    return sql_inserts

# Fungsi untuk menulis pernyataan SQL ke file
def write_sql_to_file(sql_queries, output_file):
    with open(output_file, 'w') as file:
        for sql_query in sql_queries:
            file.write(sql_query + '\n')

# Folder tempat file CSV disimpan
csv_folder = 'csv'

# Membuat pernyataan SQL CREATE TABLE dan INSERT dari setiap file CSV di dalam folder
sql_queries = []
for file_name in os.listdir(csv_folder):
    if file_name.endswith('.csv'):
        file_path = os.path.join(csv_folder, file_name)
        sql_queries.append(generate_sql_create_table(file_path))
        sql_queries.extend(generate_sql_insert_from_csv(file_path))

# Menulis pernyataan SQL ke file
output_file = 'init.sql'
write_sql_to_file(sql_queries, output_file)

print(f"Pernyataan SQL telah ditulis ke '{output_file}'.")


Pernyataan SQL telah ditulis ke 'init.sql'.


In [12]:
import os
import csv

# Fungsi untuk membaca struktur tabel dari file CSV dan menghasilkan pernyataan SQL CREATE TABLE
def generate_sql_create_table(csv_file):
    # Membaca nama file tanpa ekstensi
    table_name = os.path.splitext(os.path.basename(csv_file))[0]
    
    # Membaca struktur tabel dari file CSV
    with open(csv_file, 'r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        # Menggunakan struktur kolom pertama dalam CSV sebagai referensi untuk struktur tabel
        columns = reader.fieldnames
        # Menginisialisasi tipe data kolom dengan VARCHAR sebagai default
        column_types = {column: 'VARCHAR(255)' for column in columns}
        # Memeriksa sampel data untuk menentukan tipe data kolom yang sesuai
        sample_data = next(reader)
        for column, data in sample_data.items():
            # Jika data dapat diubah menjadi integer tanpa desimal, itu adalah tipe data INTEGER
            try:
                int_data = int(data)
                if int_data == float(data):
                    column_types[column] = 'INTEGER'
            except ValueError:
                pass
            # Jika data dapat diubah menjadi float, itu adalah tipe data DECIMAL
            try:
                float_data = float(data)
                if '.' in data:
                    column_types[column] = 'DECIMAL(10, 2)'
            except ValueError:
                pass
    
    # Membuat pernyataan SQL CREATE TABLE
    sql_create_table = f"CREATE TABLE IF NOT EXISTS {table_name} (\n"
    for column, data_type in column_types.items():
        sql_create_table += f"    {column} {data_type},\n"
    # Menghapus koma ekstra di akhir
    sql_create_table = sql_create_table.rstrip(',\n')
    sql_create_table += "\n);\n"
    
    return sql_create_table

# Fungsi untuk menulis pernyataan SQL ke file
def write_sql_to_file(sql_queries, output_file):
    with open(output_file, 'w') as file:
        for sql_query in sql_queries:
            file.write(sql_query + '\n')

# Folder tempat file CSV disimpan
csv_folder = 'csv'

# Membuat pernyataan SQL CREATE TABLE dari setiap file CSV di dalam folder
sql_queries = []
for file_name in os.listdir(csv_folder):
    if file_name.endswith('.csv'):
        file_path = os.path.join(csv_folder, file_name)
        sql_queries.append(generate_sql_create_table(file_path))

# Menulis pernyataan SQL ke file
output_file = 'init.sql'
write_sql_to_file(sql_queries, output_file)

print(f"Pernyataan SQL CREATE TABLE telah ditulis ke '{output_file}'.")


Pernyataan SQL CREATE TABLE telah ditulis ke 'init.sql'.


In [13]:
import psycopg2
from psycopg2 import Error

# Fungsi untuk menjalankan file SQL
def execute_sql_file(file_path, conn):
    try:
        with conn.cursor() as cursor:
            with open(file_path, 'r') as sql_file:
                sql_commands = sql_file.read()
                cursor.execute(sql_commands)
                conn.commit()
                print(f"File SQL {file_path} berhasil dijalankan.")
    except Error as e:
        print(f"Error: {e}")
        conn.rollback()

# Fungsi untuk membuat koneksi ke database PostgreSQL
def create_connection(db_name, db_user, db_password, db_host, db_port):
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port
        )
        print("Koneksi ke PostgreSQL berhasil")
        return connection
    except Error as e:
        print(f"Error: {e}")
        return None

# Konfigurasi koneksi ke database
db_name = "db_dskola"
db_user = "postgres"
db_password = "12345678"
db_host = "localhost"
db_port = "5434"

# Path file SQL
sql_file_path = "init.sql"

# Membuat koneksi ke database
connection = create_connection(db_name, db_user, db_password, db_host, db_port)

if connection:
    # Menjalankan file SQL
    execute_sql_file(sql_file_path, connection)

    # Menutup koneksi database
    connection.close()


Koneksi ke PostgreSQL berhasil
File SQL init.sql berhasil dijalankan.


In [15]:
import os
import csv
import psycopg2
from psycopg2 import Error

# Fungsi untuk membaca struktur tabel dari file CSV dan menghasilkan pernyataan SQL CREATE TABLE
def generate_sql_create_table(csv_file):
    table_name = os.path.splitext(os.path.basename(csv_file))[0]
    
    with open(csv_file, 'r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        columns = reader.fieldnames
        column_types = {column: 'VARCHAR(255)' for column in columns}
        sample_data = next(reader)
        for column, data in sample_data.items():
            try:
                int_data = int(data)
                if int_data == float(data):
                    column_types[column] = 'INTEGER'
            except ValueError:
                pass
            try:
                float_data = float(data)
                if '.' in data:
                    column_types[column] = 'DECIMAL(10, 2)'
            except ValueError:
                pass
    
    sql_create_table = f"CREATE TABLE IF NOT EXISTS {table_name} (\n"
    for column, data_type in column_types.items():
        sql_create_table += f"    {column} {data_type},\n"
    sql_create_table = sql_create_table.rstrip(',\n')
    sql_create_table += "\n);\n"
    
    return sql_create_table

# Fungsi untuk membaca data dari file CSV dan menghasilkan pernyataan SQL INSERT satu per satu
def generate_sql_insert_from_csv(csv_file):
    table_name = os.path.splitext(os.path.basename(csv_file))[0]
    
    with open(csv_file, 'r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        rows = [row for row in reader]
    
    sql_inserts = []
    for row in rows:
        columns = ', '.join(row.keys())
        values = ', '.join([f"'{value}'" for value in row.values()])
        sql_insert = f"INSERT INTO {table_name} ({columns}) VALUES ({values});"
        sql_inserts.append(sql_insert)
    
    return sql_inserts

# Fungsi untuk menulis pernyataan SQL ke file
def write_sql_to_file(sql_queries, output_file):
    with open(output_file, 'w') as file:
        for sql_query in sql_queries:
            file.write(sql_query + '\n')

# Fungsi untuk menjalankan pernyataan SQL
def execute_sql_statements(sql_statements, connection):
    try:
        cursor = connection.cursor()
        for sql in sql_statements:
            try:
                cursor.execute(sql)
                connection.commit()
            except Exception as e:
                print(f"Error executing query: {sql}")
                print(e)
                connection.rollback()
    except Exception as e:
        print(f"Error: {e}")
    finally:
        cursor.close()

# Folder tempat file CSV disimpan
csv_folder = 'csv'

# Koneksi ke database
connection = psycopg2.connect(
    database="db_dskola",
    user="postgres",
    password="12345678",
    host="localhost",
    port="5434"
)

# Membuat pernyataan SQL CREATE TABLE dan INSERT dari setiap file CSV di dalam folder
sql_create_queries = []
sql_insert_queries = []
for file_name in os.listdir(csv_folder):
    if file_name.endswith('.csv'):
        file_path = os.path.join(csv_folder, file_name)
        sql_create_queries.append(generate_sql_create_table(file_path))
        sql_insert_queries.extend(generate_sql_insert_from_csv(file_path))

# Menulis pernyataan SQL CREATE TABLE ke file
output_create_file = 'create_tables.sql'
write_sql_to_file(sql_create_queries, output_create_file)
print(f"Pernyataan SQL CREATE TABLE telah ditulis ke '{output_create_file}'.")

# Menjalankan pernyataan SQL CREATE TABLE
execute_sql_statements(sql_create_queries, connection)

# Menjalankan pernyataan SQL INSERT satu per satu
execute_sql_statements(sql_insert_queries, connection)

# Menutup koneksi
connection.close()


Pernyataan SQL CREATE TABLE telah ditulis ke 'create_tables.sql'.
Error executing query: INSERT INTO categories (categoryID, categoryName, description, picture) VALUES ('1', 'Beverages', 'Soft drinks coffees teas beers and ales', '0x151C2F00020000000D000E0014002100FFFFFFFF4269746D617020496D616765005061696E742E5069637475726500010500000200000007000000504272757368000000000000000000A0290000424D98290000000000005600000028000000AC00000078000000010004000000000000000000880B0000880B0000080000');
value too long for type character varying(255)

Error executing query: INSERT INTO categories (categoryID, categoryName, description, picture) VALUES ('2', 'Condiments', 'Sweet and savory sauces relishes spreads and seasonings', '0x151C2F00020000000D000E0014002100FFFFFFFF4269746D617020496D616765005061696E742E5069637475726500010500000200000007000000504272757368000000000000000000A0290000424D98290000000000005600000028000000AC00000078000000010004000000000000000000880B0000880B0000080000');
value too long for 

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os

# Informasi koneksi PostgreSQL
db_user = 'postgres'
db_password = '12345678'
db_host = 'localhost'
db_port = '5434'
db_name = 'db_dskola'

# Membuat engine untuk koneksi ke PostgreSQL
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Path ke folder yang berisi file CSV
csv_folder = 'csv'

# Loop melalui setiap file dalam folder CSV
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        # Path lengkap ke file CSV
        csv_file = os.path.join(csv_folder, filename)
        
        # Baca data dari file CSV
        df = pd.read_csv(csv_file)
        
        # Nama tabel, di sini kita menggunakan nama file (tanpa ekstensi) sebagai nama tabel
        table_name = os.path.splitext(filename)[0]
        
        # Memasukkan data ke dalam tabel di PostgreSQL
        df.to_sql(table_name, engine, if_exists='replace', index=False)
        
        print(f"Data dari {filename} berhasil dimuat ke PostgreSQL sebagai tabel {table_name}!")

print("Semua data berhasil dimuat ke PostgreSQL!")


Data dari categories.csv berhasil dimuat ke PostgreSQL sebagai tabel categories!
Data dari customers.csv berhasil dimuat ke PostgreSQL sebagai tabel customers!
Data dari employees.csv berhasil dimuat ke PostgreSQL sebagai tabel employees!
Data dari employee_territories.csv berhasil dimuat ke PostgreSQL sebagai tabel employee_territories!
Data dari orders.csv berhasil dimuat ke PostgreSQL sebagai tabel orders!
Data dari order_details.csv berhasil dimuat ke PostgreSQL sebagai tabel order_details!
Data dari products.csv berhasil dimuat ke PostgreSQL sebagai tabel products!
Data dari regions.csv berhasil dimuat ke PostgreSQL sebagai tabel regions!
Data dari shippers.csv berhasil dimuat ke PostgreSQL sebagai tabel shippers!
Data dari suppliers.csv berhasil dimuat ke PostgreSQL sebagai tabel suppliers!
Data dari territories.csv berhasil dimuat ke PostgreSQL sebagai tabel territories!
Semua data berhasil dimuat ke PostgreSQL!


In [17]:
pip install sqlalchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 330.3 kB/s eta 0:00:07
   - -------------------------------------- 0.1/2.1 MB 409.6 kB/s eta 0:00:05
   - -------------------------------------- 0.1/2.1 MB 476.3 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/2.1 MB 516.7 kB/s eta 0:00:04
   -- ------------------------------------- 0.2/2.1 MB 541.0 kB/s eta 0:00:04
   --- ------------------------------------ 0.2/2.1 MB 557.9 kB/s eta 0:00:04
   --- ------------------------------------ 0.2/2.1 MB 541.9 kB/s eta 0:00:04
   ---- ----------------------------------- 0.2/2.1 MB 533.8 kB/s eta 0:00:04
   ---- ----------------------------------- 0.3/2.1 MB 542.5 kB/s eta 0:00:04
   ----- ---------------------------------- 0.3/2.1 MB 536.9 kB/s eta 0:00:04
   ----- ---------------------------------- 0.3/2.1 MB 542.9 kB/s eta 0:00:04
   -